In [ ]:
# 对于三角形ABC，已知 R_AB, R_BC, A_ABC,得到相应的笛卡尔坐标
# For triangle ABC, given R_ AB, R_BC, A_ABC, Obtain the corresponding Cartesian coordinates
import numpy as np
def jacobian_to_cartesian_three_atom_modified(r12, r23, theta_deg):
    """
    Convert three-atom Jacobian coordinates to Cartesian coordinates with atom 1 as the origin 
    and atom 2 as the point on the axis.
    
    Parameters:
        r12 (float): Bond length between atom 1 and atom 2.
        r23 (float): Bond length between atom 2 and atom 3.
        theta_deg (float): Bond angle between atoms 1-2-3 in degrees.
    
    Returns:
        numpy.ndarray: Cartesian coordinates of atoms 1, 2, and 3 in a (3, 3) array.
    """
    # Convert the angle from degrees to radians
    theta = np.radians(theta_deg)
    
    # Define the Cartesian coordinates of atom 1 as the origin
    atom1_coords = np.array([0, 0, 0])
    
    # Calculate the x, y, z coordinates of atom 2 relative to atom 1
    atom2_x = r12
    atom2_y = 0
    atom2_coords = np.array([atom2_x, atom2_y, 0])
    
    # Calculate the x, y, z coordinates of atom 3 relative to atom 2
    atom3_x = r23 * np.cos(theta)
    atom3_y = r23 * np.sin(theta)
    atom3_coords = np.array([atom2_x + atom3_x, atom3_y, 0])
    
    # Translate the coordinates of atoms 2 and 3 relative to atom 1
    atom2_coords += atom1_coords
    atom3_coords += atom1_coords
    
    # Combine the coordinates of atoms 1, 2, and 3 into a (3, 3) array
    cartesian_coords = np.vstack((atom1_coords, atom2_coords, atom3_coords))
    print('input', r12, r23, theta_deg)
    return cartesian_coords

# Test the modified function
jacobian_coords_modified = jacobian_to_cartesian_three_atom_modified(1.0, 1.0, 90)
jacobian_coords_modified

# import matplotlib.pyplot as plt

# # Existing code
# # ...

# # Test the modified function
# jacobian_coords_modified = jacobian_to_cartesian_three_atom_modified(1.0, 1.0, 90)

# # Plotting
# plt.scatter(jacobian_coords_modified[:, 0], jacobian_coords_modified[:, 1])
# plt.text(jacobian_coords_modified[0, 0], jacobian_coords_modified[0, 1], 'Atom 1')
# plt.text(jacobian_coords_modified[1, 0], jacobian_coords_modified[1, 1], 'Atom 2')
# plt.text(jacobian_coords_modified[2, 0], jacobian_coords_modified[2, 1], 'Atom 3')

# plt.xlabel('X')
# plt.ylabel('Y')
# plt.grid(True)
# plt.show()


In [ ]:
# 将三原子的雅可比坐标转换为笛卡尔坐标
# Convert three-atom Jacobi coordinates to Cartesian coordinates.
import numpy as np

def jacobi_to_cartesian(r_AB, r_C, theta_deg):
    """
    Convert three-atom Jacobi coordinates to Cartesian coordinates.

    Parameters:
        r_AB (float): Distance between atoms A and B.
        r_C (float): Distance between the center of mass of AB and atom C.
        theta_deg (float): Angle between AB and C in degrees.

    Returns:
        numpy.ndarray: Cartesian coordinates of atoms A, B, and C.
    """
    # Convert the angle from degrees to radians
    theta = np.radians(theta_deg)
    
    # Define positions of atoms A and B in Jacobi coordinates
    x_A, y_A, z_A = 0, 0, 0
    x_B, y_B, z_B = r_AB, 0, 0
    
    # Calculate the position of the center of mass of AB
    x_AB_cm = (x_A + x_B) / 2
    y_AB_cm = (y_A + y_B) / 2
    z_AB_cm = (z_A + z_B) / 2
    
    # Calculate the position of atom C in the plane of AB
    x_C = x_AB_cm + r_C * np.cos(theta)
    y_C = y_AB_cm + r_C * np.sin(theta)
    z_C = z_AB_cm
    
    return np.array([[x_A, y_A, z_A],
                     [x_B, y_B, z_B],
                     [x_C, y_C, z_C]])

# Example usage:
r_AB = 2.0
r_C = 1.0
theta_deg = 90  
cartesian_coords = jacobi_to_cartesian(r_AB, r_C, theta_deg)
print("Cartesian coordinates:")
print(cartesian_coords)

In [2]:
# 输出转换坐标 # Output Conversion Coordinates
import numpy as np

def angle_between_vectors(v1, v2):
    v1_u = v1 / np.linalg.norm(v1)
    v2_u = v2 / np.linalg.norm(v2)
    return np.degrees(np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)))

def centroid(m1, v1, m2, v2):
    return (m1 * v1 + m2 * v2) / (m1 + m2)

def convert_jacobian_coordinates(m_A, m_B, m_C, d_AB, d_QC, theta_AB_CQ_deg):
    vect_A = np.array([0, 0])
    vect_B = np.array([d_AB, 0])
    theta_AB_CQ_rad = np.radians(theta_AB_CQ_deg)
    
    vect_QAB = centroid(m_A, vect_A, m_B, vect_B)
    vect_QC = np.array([d_QC * np.cos(theta_AB_CQ_rad), d_QC * np.sin(theta_AB_CQ_rad)])
    vect_C = vect_QAB + vect_QC
    
    d_AC = np.hypot(*(vect_C - vect_A))
    d_BC = np.hypot(*(vect_C - vect_B))
    
    vect_QAC = centroid(m_A, vect_A, m_C, vect_C)
    vect_QB = vect_B - vect_QAC
    d_QB = np.hypot(*vect_QB)
    theta_AC_BQ_deg = angle_between_vectors(vect_QB, vect_C)
    
    vect_QBC = centroid(m_B, vect_B, m_C, vect_C)
    vect_QA = vect_A - vect_QBC
    d_QA = np.hypot(*vect_QA)
    theta_BC_AQ_deg = angle_between_vectors(vect_QA, vect_C - vect_B)
    
    return (d_AC, d_QB, theta_AC_BQ_deg), (d_BC, d_QA, theta_BC_AQ_deg)

print(convert_jacobian_coordinates(1.0, 1.0, 1.0, 2.0, 1.0, 90.0))

((1.4142135623730951, 1.5811388300841898, 63.43494882292202), (1.4142135623730951, 1.5811388300841898, 63.43494882292202))


In [3]:
# 指定输出坐标 # Specify output coordinates
import numpy as np

def angle_between_vectors(v1, v2):
    v1_u = v1 / np.linalg.norm(v1)
    v2_u = v2 / np.linalg.norm(v2)
    return np.degrees(np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)))

def centroid(m1, v1, m2, v2):
    return (m1 * v1 + m2 * v2) / (m1 + m2)

def convert_jacobian_coordinates(m_A, m_B, m_C, d_AB, d_QC, theta_AB_CQ_deg, return_for='B'):
    vect_A = np.array([0, 0])
    vect_B = np.array([d_AB, 0])
    theta_AB_CQ_rad = np.radians(theta_AB_CQ_deg)
    
    vect_QAB = centroid(m_A, vect_A, m_B, vect_B)
    vect_QC = np.array([d_QC * np.cos(theta_AB_CQ_rad), d_QC * np.sin(theta_AB_CQ_rad)])
    vect_C = vect_QAB + vect_QC
    
    d_AC = np.hypot(*(vect_C - vect_A))
    d_BC = np.hypot(*(vect_C - vect_B))
    
    vect_QAC = centroid(m_A, vect_A, m_C, vect_C)
    vect_QB = vect_B - vect_QAC
    d_QB = np.hypot(*vect_QB)
    theta_AC_BQ_deg = angle_between_vectors(vect_QB, vect_C)
    
    vect_QBC = centroid(m_B, vect_B, m_C, vect_C)
    vect_QA = vect_A - vect_QBC
    d_QA = np.hypot(*vect_QA)
    theta_BC_AQ_deg = angle_between_vectors(vect_QA, vect_C - vect_B)
    
    if return_for == 'B':
        return d_AC, d_QB, theta_AC_BQ_deg
    elif return_for == 'C':
        return d_BC, d_QA, theta_BC_AQ_deg
    else:
        raise ValueError("return_for must be either 'B' or 'C'")

print(convert_jacobian_coordinates(1.0, 1.0, 1.0, 2.0, 1.0, 90.0, return_for='B'))
print(convert_jacobian_coordinates(1.0, 1.0, 1.0, 2.0, 1.0, 90.0, return_for='C'))

(1.4142135623730951, 1.5811388300841898, 63.43494882292202)
(1.4142135623730951, 1.5811388300841898, 63.43494882292202)
